In [87]:
import tensorflow as tf

In [88]:
import keras_tuner as kt


In [89]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [90]:
#
# Import Keras modules
#
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import to_categorical

from tensorflow.keras import Model
from tensorflow.keras import Sequential
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import MeanSquaredLogarithmicError

In [91]:
from datetime import datetime
now = datetime.now()
logdir_tuner = "tf_logs/" + now.strftime("%Y%m%d-%H%M%S") + "/" + "Tuner"
logdir_tensorboard = "tf_logs/" + now.strftime("%Y%m%d-%H%M%S") + "/" + "TensorBoard"

In [92]:
#
# Load the reg boston housing dataset
#
boston = datasets.load_boston()
X = boston.data
y = boston.target
#
# Split data
#
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [93]:
# boston.feature_names

In [94]:
# y

In [95]:
# df = pd.DataFrame(X,y)
# df

In [96]:
pd.Series(y_train).unique().size

195

In [97]:
X_train.shape[1]

13

In [98]:
# def regression_model(hp):
#     inputs = Input(shape = X_train.shape[1])
#     hidden = inputs
    
#     hp_layers = hp.Int("Dense_layers", min_value=1, max_value=3, step=1, default=3)
#     hp_dropout = hp.Choice("Dropout_layer", [0.1, 0.2, 0.3], default=0.2)
        
#     for i in range(hp_layers):
#         hidden = layers.Dense(hp.Choice("dense_units_" + str(i), [16, 32, 64], default=16),
#                               activation=hp.Choice("activation_" + str(i), ['relu', 'tanh']))(hidden)
#         hidden = layers.Dropout(hp_dropout)(hidden)
    
#     outputs = layers.Dense(pd.Series(y_train).unique().size, 
#                            activation="softmax")(hidden)
    
#     model = tf.keras.Model(inputs, outputs)

#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(
#             hp.Choice("learning_rate", values=[1e-2, 1e-3])
#         ),
#         loss="categorical_crossentropy",
#         metrics=["accuracy"]
#     )
#     return model

In [99]:
# from kerastuner import HyperModel

# class RegressionHyperModel(HyperModel):
#     def __init__(self, input_shape):
#         self.input_shape = input_shape
#     def build(self, hp):
#         model = Sequential()
#         model.add(
#             layers.Dense(
#                 units=hp.Int('units', 8, 64, 4, default=8),
#                 activation=hp.Choice('dense_activation', values=['relu', 'tanh', 'sigmoid'], default='relu'),
#                 input_shape=input_shape
#             )
#         )
        
#         model.add(
#             layers.Dense(
#                 units=hp.Int('units', 16, 64, 4, default=16),
#                 activation=hp.Choice('dense_activation',values=['relu', 'tanh', 'sigmoid'], default='relu')
#             )
#         )
        
#         model.add(
#             layers.Dropout( hp.Float('dropout', min_value=0.0, max_value=0.1, default=0.005,step=0.01)
#             )
#         )
        
#         model.add(layers.Dense(1))
        
#         model.compile(
#             optimizer='rmsprop',loss='mse',metrics=['mse']
#         )
        
#         return model

# input_shape = (x_train.shape[1],)
# hypermodel = RegressionHyperModel(input_shape)

# tuner_rs = RandomSearch(
#             hypermodel,
#             objective='mse',
#             seed=42,
#             max_trials=10,
#             executions_per_trial=2)

# tuner_rs.search(x_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=0)

# best_model = tuner_rs.get_best_models(num_models=1)[0]
# loss, mse = best_model.evaluate(x_test_scaled, y_test)

In [100]:
msle = MeanSquaredLogarithmicError()


def build_model(hp):
    model = tf.keras.Sequential()

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units1 = hp.Int('units1', min_value=32, max_value=512, step=32)
    hp_units2 = hp.Int('units2', min_value=32, max_value=512, step=32)
    hp_units3 = hp.Int('units3', min_value=32, max_value=512, step=32)
    model.add(Dense(units=hp_units1, activation='relu'))
    model.add(tf.keras.layers.Dense(units=hp_units2, activation='relu'))
    model.add(tf.keras.layers.Dense(units=hp_units3, activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='linear'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(
                      optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                      loss=msle,
                      metrics=[msle]
                    )

    return model

In [101]:
# HyperBand algorithm from keras tuner
tuner = kt.Hyperband(
                        build_model,
                        objective='val_mean_squared_logarithmic_error',
                        max_epochs=10,
                        directory='keras_tuner_dir',
                        project_name='keras_tuner_demo'
                    )

INFO:tensorflow:Reloading Oracle from existing project keras_tuner_dir\keras_tuner_demo\oracle.json
INFO:tensorflow:Reloading Tuner from keras_tuner_dir\keras_tuner_demo\tuner0.json


In [102]:
tuner.search(
                X_train, 
                y_train, 
                epochs=10, 
                validation_split=0.2, 
                callbacks=[tf.keras.callbacks.TensorBoard(logdir_tensorboard)]
            )

INFO:tensorflow:Oracle triggered exit


In [103]:
X_train.shape ,y_train.shape

((354, 13), (354,))

In [104]:
best_model = tuner.get_best_models()[0]
best_model.build(X_train.shape)
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (354, 160)                2240      
                                                                 
 dense_1 (Dense)             (354, 512)                82432     
                                                                 
 dense_2 (Dense)             (354, 320)                164160    
                                                                 
 dense_3 (Dense)             (354, 1)                  321       
                                                                 
Total params: 249,153
Trainable params: 249,153
Non-trainable params: 0
_________________________________________________________________


In [105]:
best_model.fit(
                    X_train, 
                    y_train,
                    epochs=10,
                    batch_size=64
                )

Epoch 1/10
6/6 [==============================] - 0s 5ms/step - loss: 0.1058 - mean_squared_logarithmic_error: 0.1062
Epoch 2/10
6/6 [==============================] - 0s 4ms/step - loss: 0.1008 - mean_squared_logarithmic_error: 0.1054
Epoch 3/10
6/6 [==============================] - 0s 4ms/step - loss: 0.1049 - mean_squared_logarithmic_error: 0.1113
Epoch 4/10
6/6 [==============================] - 0s 3ms/step - loss: 0.1128 - mean_squared_logarithmic_error: 0.1123
Epoch 5/10
6/6 [==============================] - 0s 3ms/step - loss: 0.1102 - mean_squared_logarithmic_error: 0.1130
Epoch 6/10
6/6 [==============================] - 0s 3ms/step - loss: 0.1004 - mean_squared_logarithmic_error: 0.1040
Epoch 7/10
6/6 [==============================] - 0s 3ms/step - loss: 0.0982 - mean_squared_logarithmic_error: 0.0938
Epoch 8/10
6/6 [==============================] - 0s 3ms/step - loss: 0.0984 - mean_squared_logarithmic_error: 0.0986
Epoch 9/10
6/6 [==============================] - 0s 3ms

In [106]:
# mean squared logarithmic error
msle(y_test, best_model.predict(X_test)).numpy()

5/5 [==============================] - 0s 1ms/step


0.21758471

In [107]:
import tensorboard

In [108]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [109]:
logdir_tensorboard

'tf2_logs/20220424-002356/TensorBoard'

In [110]:
%tensorboard --logdir "$logdir_tensorboard"

In [113]:
%tensorboard --logdir 'tf2_logs/20220420-151335/TensorBoard'

Reusing TensorBoard on port 6006 (pid 15364), started 3 days, 8:47:06 ago. (Use '!kill 15364' to kill it.)

In [80]:
# !pip uninstall tb-nightly tensorboardX tensorboard

In [ ]:
# !pip install tensorboard

In [52]:
!pip list 

Package                            Version
---------------------------------- --------------------
absl-py                            1.0.0
alabaster                          0.7.12
anaconda-client                    1.9.0
anaconda-navigator                 2.1.1
anaconda-project                   0.10.1
anyio                              2.2.0
appdirs                            1.4.4
argh                               0.26.2
argon2-cffi                        20.1.0
arrow                              0.13.1
asn1crypto                         1.4.0
astroid                            2.6.6
astropy                            4.3.1
astunparse                         1.6.3
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              21.2.0
autopep8                           1.5.7
Babel                              2.9.1
backcall                           0.2.0
backports.functools-lru-cache      1.6.4
backports.shutil-get-terminal-size 

In [54]:
!pip install tensorflow

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\Lamyaa Zayed\\anaconda3\\Scripts\\tensorboard.exe' -> 'C:\\Users\\Lamyaa Zayed\\anaconda3\\Scripts\\tensorboard.exe.deleteme'
Consider using the `--user` option or check the permissions.




  Using cached tensorflow-2.8.0-cp39-cp39-win_amd64.whl (438.0 MB)
  Using cached libclang-13.0.0-py2.py3-none-win_amd64.whl (13.9 MB)
  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
